In [ ]:
# Run Scraper Bot

In [1]:
# from blogscraper.basebot import BaseBot
# from blogscraper.LanguageModel import LanguageModel
from staysaasy_bot import StaySaasyBot
from eugeneyan_bot import EugeneYanBot
from lenny_bot import LennyBot
from app import UserInterface
from language_model import LanguageModel
import config
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md
import re

In [2]:
ui = UserInterface(logfile_path="../logs/logs.log") # need modified path for invoking from within directory

In [3]:
ui.read_database()

2024-02-08 05:58:30,466 INFO Loaded database from file


In [4]:
config.BLOGS

{'Lenny': {'base_url': 'https://www.lennysnewsletter.com/t/',
  'bot': 'lennybot'}}

In [5]:
bot = LennyBot(blog_name='Lenny', database=ui.database, debug=True)

In [6]:
# bot.get_blog_pages()

In [7]:
pages = requests.get(config.BLOGS.get('Lenny').get("base_url"),timeout=100)

In [8]:
soup = BeautifulSoup(pages.content, 'html.parser')
print(soup)

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="24usqpep0ejc5w6hod3dulxwciwp0djs6c6ufp96av3t4whuxovj72wfkdjxu82yacb7430qjm8adbd5ezlt4592dq4zrvadcn9j9n-0btgdzpiojfzno16-fnsnu7xd" name="norton-safeweb-site-verification"/>
<link href="https://substackcdn.com" rel="preconnect"/>
<link as="style" href="https://substackcdn.com/bundle/theme/main.02a5cba6affcac40ef08.css" rel="preload"/>
<link as="style" href="https://substackcdn.com/bundle/main.b871798ce17b64a99457.css" rel="preload"/>
<link as="font" crossorigin="" href="https://fonts.gstatic.com/s/spectral/v13/rnCr-xNNww_2s0amA9M5knjsS_ul.woff2" rel="preload"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=0, viewport-fit=cover" name="viewport"/>
<meta content="Lenny Rachitsky" name="author"/>
<meta content="https://www.lennysnewsletter.com/t" property="og:url"/>
<title data-preact-helmet="">Lenny's Newsletter | Lenny Rachitsky | Substack</title>
<meta content="article" da

In [9]:
link_list = [link.get('href') for link in soup.find_all('a')]
link_list_filtered = [link for link in link_list if isinstance(link, str) and link[32:35] == '/t/']

In [10]:
set(link_list_filtered)

{'https://www.lennysnewsletter.com/t/benchmarks',
 'https://www.lennysnewsletter.com/t/career',
 'https://www.lennysnewsletter.com/t/growth',
 'https://www.lennysnewsletter.com/t/product'}

In [11]:
all_links = []
for link in link_list_filtered:
    pages = requests.get(link,timeout=100)
    soup = BeautifulSoup(pages.content, 'html.parser')
    link_list = [link.get('href') for link in soup.find_all('a')]
    link_list_filtered = [link for link in link_list if isinstance(link, str) and "lennysnewsletter" in link and "comment" not in link and "subscribe" not in link]
    all_links.extend(link_list_filtered)

all_links = list(set(all_links))

In [13]:
all_links

['https://www.lennysnewsletter.com/p/how-to-pass-any-first-round-interview',
 'https://www.lennysnewsletter.com/p/how-to-get-promoted',
 'https://www.lennysnewsletter.com/t/career',
 'https://www.lennysnewsletter.com/p/doing-a-good-job-product-manager',
 'https://www.lennysnewsletter.com/p/how-the-most-successful-b2b-startups',
 'https://www.lennysnewsletter.com/p/how-ramp-builds-product',
 'https://www.lennysnewsletter.com/p/managing-up',
 'https://www.lennysnewsletter.com/t/benchmarks',
 'https://www.lennysnewsletter.com/p/how-to-validate-your-b2b-startup',
 'https://www.lennysnewsletter.com/p/lessons-on-building-a-viral-consumer',
 'https://www.lennysnewsletter.com/p/lessons-on-building-product-sense',
 'https://www.lennysnewsletter.com/p/lessons-from-scaling-spotify-the',
 'https://www.lennysnewsletter.com/p/how-notion-builds-product',
 'https://www.lennysnewsletter.com/p/what-working-at-figma-taught-me-about',
 'https://www.lennysnewsletter.com/p/the-magic-loop',
 'https://www.len

In [14]:
base = config.BLOGS.get('Lenny').get('base_url')

In [ ]:
link_list_appended = [base[:-5] + link for link in link_list_filtered]

In [ ]:
link_list_appended[0]

In [15]:
# page = requests.get(link_list_appended[0], timeout=100)
page = requests.get(all_links[0], timeout=100)

In [16]:
soup = BeautifulSoup(page.content, 'html.parser')

In [17]:
print(re.sub(r'(?:\r?\n|\r){2,}', "", soup.get_text()))

How to pass any first-round interview (even in a terrible talent market)Lenny's NewsletterSubscribeSign inShare this postHow to pass any first-round interview (even in a terrible talent market)www.lennysnewsletter.comCopy linkFacebookEmailNoteOtherHow to pass any first-round interview (even in a terrible talent market)Learn how to get that strong yes without burning yourself out with interview prepCoach ErikaJul 4, 2023316Share this postHow to pass any first-round interview (even in a terrible talent market)www.lennysnewsletter.comCopy linkFacebookEmailNoteOther23Share👋 Hey, Lenny here! Welcome to this month’s ✨ free edition ✨ of Lenny’s Newsletter. Each week I tackle reader questions about building product, driving growth, and accelerating your career.If you’re not a subscriber, here’s what you missed this month:The unconventional Palantir principles that catalyzed a generation of startupsWhat 5 years at Reddit taught us about building for a highly opinionated user baseHow a traumatic

In [18]:
soup.title.string.replace(":", " ").replace("/", " ")

'How to pass any first-round interview (even in a terrible talent market)'

In [19]:
lm = LanguageModel()

In [20]:
cleaned_text, usages = lm.clean_blog(soup.get_text())

2024-02-08 05:59:49,562 INFO Text broken into 2 chunks
2024-02-08 06:00:43,735 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-08 06:00:45,997 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [21]:
print(cleaned_text)

Summary:
In this article, Coach Erika shares a step-by-step guide for passing a first-round interview in tech, even in a difficult talent market. She provides practical frameworks, practice questions, and a daily schedule to help job seekers make the most of their available prep time.

Article:
How to pass any first-round interview (even in a terrible talent market)

Lenny's NewsletterSubscribeSign inShare this postHow to pass any first-round interview (even in a terrible talent market)www.lennysnewsletter.comCopy linkFacebookEmailNoteOtherHow to pass any first-round interview (even in a terrible talent market)Learn how to get that strong yes without burning yourself out with interview prepCoach ErikaJul 4, 2023316Share this postHow to pass any first-round interview (even in a terrible talent market)www.lennysnewsletter.comCopy linkFacebookEmailNoteOther23Share👋 Hey, Lenny here! Welcome to this month’s ✨ free edition ✨ of Lenny’s Newsletter. Each week I tackle reader questions about bu

In [ ]:
usages